In [ ]:
pip install matplotlib

In [ ]:
import time
import random
import threading
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pygame
import matplotlib.pyplot as plt

# Initialize Pygame mixer for playing sounds
pygame.mixer.init()

# Function to get real sensor data (replace with actual implementation)
def get_real_sensor_data():
    # In a real-world scenario, this function would obtain actual sensor readings
    temperature = 98.6 + random.uniform(-0.5, 0.5)
    blood_pressure = 120 + random.uniform(-5, 5)
    return temperature, blood_pressure

# Function for real-time data collection and plotting
def collect_data(model):
    # Lists to store historical sensor readings
    temperature_history = []
    blood_pressure_history = []

    # Create figure and axes for subplots
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 6))

    while True:
        temperature, blood_pressure = get_real_sensor_data()

        # Use the trained model for prediction
        status = model.predict(np.array([[temperature, blood_pressure]]))[0]

        print(f'Temperature: {temperature:.2f}°F, Blood Pressure: {blood_pressure:.2f} mmHg, Status: {"Normal" if status == 1 else "Abnormal"}')

        # Append readings to history lists
        temperature_history.append(temperature)
        blood_pressure_history.append(blood_pressure)

        # Update real-time plots
        axes[0].clear()
        axes[1].clear()

        axes[0].plot(temperature_history, label='Temperature', color='blue')
        axes[0].set_title('Temperature History')
        axes[0].set_xlabel('Time')
        axes[0].set_ylabel('Temperature (°F)')

        axes[1].plot(blood_pressure_history, label='Blood Pressure', color='red')
        axes[1].set_title('Blood Pressure History')
        axes[1].set_xlabel('Time')
        axes[1].set_ylabel('Blood Pressure (mmHg)')

        plt.tight_layout()
        plt.pause(0.1)  # Pause to allow the plot to update

        if status == 0:
            # If the status is abnormal, play an alarm sound
            pygame.mixer.music.load("Alarm.mp3")  # Replace with the path to your alarm sound file
            pygame.mixer.music.play()

        time.sleep(1)  # Simulate a delay to mimic real-time data collection

# Sample data for training the model
training_data = {
    'Temperature': [98.6, 99.2, 97.8, 98.5, 99.0, 97.5],
    'BloodPressure': [120, 122, 118, 119, 121, 117],
    'Normal': [1, 1, 0, 1, 1, 0]  # 1 for normal, 0 for abnormal
}

df = pd.DataFrame(training_data)

# Split the data into features and target
X_train = df[['Temperature', 'BloodPressure']]
y_train = df['Normal']

# Create and train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Create a separate thread for real-time data collection
real_time_thread = threading.Thread(target=collect_data, args=(model,))
real_time_thread.start()

# Main thread can perform other tasks or wait for real-time data collection to finish
real_time_thread.join()